# 敏捷功能验证

**MinJie Functional Verification**

![MinJie Cycle](../images/02-functional/00-intro/overal-en.png)

我们使用敏捷平台构建工作流程，如图中所示的功能验证工具链。

We use MinJie Platform to construct the workflow, which is shown as functional verification toolchain in the picture.

我们将功能验证总结为以下 4 个部分组成的循环，并为每个部分提供了工具支持：

- 测试生成：nexus-am 用于生成裸机测试；
- 错误检测：nemu 提供参考结果，difftest 作为结果比对工具；
- 保存错误上下文：lightsss 自动捕捉现场；
- 故障排查和错误修复：波形图和 chiselDB 用于定位和修复错误。

We summarize functional verification as a loop consisting of the following four parts, with tool support for each part:

- Test generation: nexus-am for generating bare-metal tests;
- Bug detection: nemu for providing golden result, difftest for result comparison;
- Preserving bug context: lightsss for automatic context capturing;
- Troubleshoot & bug fixation: waveform and chiselDB for bug locating and fixing.

我们还设计了一些小工具用来对处理器的特定部件进行单元测试，例如 Tilelink 总线测试工具 `tltest`。

We also designed some small tools for unit testing specific components of the processor, such as the Tilelink bus testing tool `tltest`.

## Why we need Minjie

我们在 ALU 模块内人为的注入了一个错误，并编译好了emu供大家测试：

We manually injected an bug into the ALU module and compiled emu for you to test:

In [ ]:
%%bash
source ../env.sh
cd ${NOOP_HOME}

$(get_asset emu-precompile/emu-alu-err) \
    -i $(get_asset workload/hello-riscv64-xs.bin ${AM_HOME}/apps/hello/build/hello-riscv64-xs.bin) \
    --no-diff \
    -C 20000 \
    2>/dev/null || true

可以看到，它没有像前一节一样正确输出“Hello XiangShan”。并且在到达 20000 周期的限制时 `pc` 为 `0xE`，这显然不是正常程序应有的行为。

We can see that it does not correctly print "Hello XiangShan" as the previous section did. And, when it reaches the 20,000-cycle limit, `pc` is `0xE`. That's clearly not the behavior of a normal program.

然而，仿真程序自己并不能知道运行“对”还是“错”，因此它可能会在执行错误的行为后永无止境的运行下去，这只是在空耗计算资源。

However, the simulation program cannot determine whether it is running correctly or incorrectly. As a result, it may continue running indefinitely after the faulty behaviour, merely wasting compute resources.

“仿真程序怎么知道自己已经出错了”，进而如何“找到错误发生的位置”，是处理器功能验证的重要问题。Minjie 工具链就是用来解决这些问题的。

"How does the simulation program know it has already encountered an error" and further "How to locate where the error occurred" are important questions in processor functional verification. Minjie toolchain is designed to solve these questions.